<a href="https://colab.research.google.com/github/fpaleixo/arbolom/blob/main/ARBoLoM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARBoLoM - ASP Revision of Boolean Logical Models

## Objective

The ARBoLoM tool aims to automate the revision of Boolean logical models, with fully Answer Set Programming-based consistency checking and repair algorithms.

## Quick testing
For quick testing, the models in the `simple_models` folder may be used. These are small, hand-made examples of hypothethical biological networks. Due to their compact nature, one can use them to perform quick tests with the scripts, or to manually verify the correctness of the results produced by said scripts.

Generally, the models will be used in the following fashion:



*   One of the models is chosen to perform tests on.
*   The `corruption.py` script can then be used to create a "wrong" version of this model.
* `conversion.py` is ran on both models, to convert them to .lp format.
* `gen_observations.py` is ran on the original .lp model, to generate observations that can be used to check the correctness of the wrong model.
* `consistency_checking.py` is then ran on the wrong .lp model, using the observations generated from the original model. This script will tell you if the model is consistent with the given observations or not.
* In case it is not, `repair.py` can be used to repair the inconsistent model.

(At a later date, a single script should be implemented to automatically convert, check the consistency of, and repair a model written in the .bnet format, given a set of observations in .lp format).

# Directory set-up & installing clingo 

## Directory set-up

**Method #1**: Connect to a Google Drive (run the two cells below)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#ARBoLoM directory setup, make sure to set it to the arbolom folder (%cd "/content/drive/.../arbolom")
%cd "/content/drive/MyDrive/FCT/5o ano/2o semestre/arbolom"

**Method #2**: Upload the arbolom zip to Colab.

 To do this, zip the downloaded arbolom directory, upload the .zip to Colab, and use the `!unzip filename.zip` command below to unzip (replacing filename with the appropriate name).

In [ ]:
!unzip arbolom-main.zip

In [ ]:
#ARBoLoM directory setup, make sure to set it to the arbolom folder (%cd ".../arbolom")
%cd "/content/arbolom-main"

## Installing clingo

In [ ]:
#Install clingo
!pip install clingo

# Script Usage Example

In the example below, we will use the scripts to corrupt an existing .bnet model using `corruption.py` and convert that model to .lp format using `conversion.py`.

We will then generate some observations from the original model in .lp format using `gen_observations.py`, which we will then use to check the consistency of the corrupted model with `consistency_checking.py`.

Lastly, after verifying that the corrupted model is inconsistent, we will use `repair.py` to obtain repairs that would render it consistent.

_____

*Note: in the future, a single `revision.py` script will be used to take a model in .bnet format, alongside a file with observations in .lp format, and automatically verify its consistency and display whatever repairs are necessary to render the model consistent, in case it is inconsistent.*


In [ ]:
#Usage: $python corruption.py -f (FILENAME) -op (OPERATIONS) -(O)p (PROBABILITY) -s (SAVE_PATH)
#Optional flags:
#-s -> Saves the output in a custom directory.
#Variables: 
#FILENAME -> Path of file inside simple models folder to corrupt.
#OPERATIONS -> A string with one (or more) specific characters, denoting which 
# corruptions to apply. These characters are 'f','e','r' and 'a'. 
# 'fera' would be the full string, representing that (f)unction change, 
# (e)dge flip, edge (r)emove and edge (a)dd will all be applied.
#O -> A character that can take one of four possible values: 'f','e','r' and 'a' 
# (followed by 'p'). -fp would change the probability of function change to 
# occur, -ep of edge removal, etc. The argument that uses this O variable is an 
# optional one.
#PROBABILITY -> A float from 0.0 to 1.0 denoting the probability of a given 
# corruption to occur. For example, -ap 0.5 would change the add edge 
# operation's probability to 50%.
#SAVE_PATH -> Path where the corrupted model will be saved in.

!python corruption.py -op era -ep 0.5 -rp 0.5 -ap 0.5 -f "simple_models/1.bnet" -s demo

In [ ]:
#Usage: $python conversion.py -f (FILENAME) -s (SAVE_PATH)
#Optional flags:
#-s -> Saves the output in a custom directory.
#Variables: 
#FILENAME -> Path of file containing Boolean model in the BCF format to 
#  convert to ASP.
#SAVE_PATH -> Path of the directory to save the converted model.

!python conversion.py -f "demo/1-corrupted-era.bnet" -s demo

In [ ]:
#Usage: $python gen_observations.py -f (FILENAME) -async -e (NUMBER OF EXPERIMENTS) -t (TIME STEPS) -s (SAVE_PATH)
#Optional flags:
#-async -> Produces the observations using asynchronous mode (default is sync).
#-s -> Saves the output in a custom directory.
#Variables:
#FILENAME -> Path of file containing Boolean model in the BCF format written in lp.
#NUMBER OF EXPERIMENTS -> The number of experiments (sets of observations) to generate.
#TIME STEPS -> The number of time steps to consider in each experiment.
#SAVE_PATH -> Path of the directory to save the generated observations.


!python gen_observations.py -f "simple_models/lp/1.lp" -t 20 -e 5 -s demo

In [ ]:
#Usage: $python consistency_checking.py -f (FILENAME) -o (OBSERVATIONS) -stable -sync -async -s (SAVE_PATH)
#Optional flags:
#-stable -> Performs consistency checking using stable state observations.
#-sync -> Performs consistency checking using synchronous observations.
#-async -> Performs consistency checking using asynchronous observations.
#-s -> Saves the output in a custom directory.
#Variables:
#FILENAME -> Path of file containing Boolean model in the BCF format written in lp.
#OBSERVATIONS -> Path of file containing observations written in lp. 
#SAVE_PATH -> Path of directory to save inconsistencies to (if any exist).

!python consistency_checking.py -f "demo/1-corrupted-era.lp" -o "demo/1-sync-obs.lp" -sync

In [ ]:
#Usage: $python repair.py -f (FILENAME) -i (INCONSISTENCIES) -stable -sync -async
#Optional flags:
#-stable -> Performs repairs using stable state observations (default).
#-sync -> Performs repairs using synchronous observations.
#-async -> Performs repairs using asynchronous observations.
#Variables:
#FILENAME -> Path of file containing Boolean model in the BCF format written in lp.
#INCONSISTENCIES -> Path of file containing inconsistencies obtained from the consistency checking phase.

!python repair.py -f "demo/1-corrupted-era.lp"  -i "demo/inconsistencies/1-corrupted-era-sync_inconsistency.lp" -sync